# GOAL: 将EXCEL转换成JSON文件


- 先将excel转换成csv文件


- 遍历读取csv文件，每个csv文件作为一个`dataframe`
- dataframe每4列作为一个单元。
- 先判断文件列表中某个钻孔文件是否存在，如果存在，跳过本次操作；如果不存在，执行下一步
- 取这6列中非空的元素。生成json格式。
- 类型检验：检验地层是否完整。去掉文本中的空格和其他特殊符号
- 如果没有通过类型检验，错误原因+钻孔名称输出到log文件



- 将坐标数据转换成GeoJSON格式
- 将每一行转换成文字
- 保留2位有效数字
- 文字去空格
- 校验一下数据类型
- 去掉文字中的双引号

```json
{
    "drill":[
        {

            "drill-name":"drill-BK199",
            "coord_x":419135.54,
            "coord_y":4872561.17,
            "elevation":987.55
        },
        {
            "stratum":[
                {
                    "strata_name":"砂土",
                    "top_m":0.0,
                    "bot_m":10.450,
                    "description":"土黄色，松散，主要由风成砂土、坡积砂土组成，顶部富含现代植物根系，底部含砾。"
                },
                {
                    "strata_name":"砂质泥岩",
                    "top_m":10.450,
                    "bot_m":45.0,
                    "description":"灰绿色，泥质结构，块状构造，砂多以粉砂为主，分布不均一，断口参差状，松软，风化易碎。"
                },
                {
                    "strata_name":"细粒砂岩",
                    "top_m":45.0,
                    "bot_m":55.5,
                    "description":"灰绿色，细粒砂状结构，厚层状构造，成分以石英为主，长石次之，云母及暗色矿物少量，分选差，泥质胶结，均匀层理，疏松，易碎。"
                },
                {
                    "strata_name":"砂砾岩",
                    "top_m":55.5,
                    "bot_m":89.9,
                    "description":"灰绿色，松散，砂多以细砂为主，含量约25%左右，砾多以英岩砾，火成岩砾组成，砾径10-20mm,多呈次棱角状，偶见圆状，泥质胶结。"
                },
                {
                    "strata_name":"砂质泥岩",
                    "top_m":89.9,
                    "bot_m":117.05,
                    "description":"灰色，泥质结构，块状构造，砂多以粉砂为主，分布不均一，断口平坦状，局部含少量炭化植物碎屑及茎叶化石，半坚硬。"
                },
                {
                    "strata_name":"细粒砂岩",
                    "top_m":117.05,
                    "bot_m":144.24,
                    "description":"灰色，细粒砂状结构，厚层状构造，成分以石英为主，长石次之，云母少量，分选中等，泥质胶结，均匀层理，夹砂质泥岩薄层，半坚硬。"
                },
                {
                    "strata_name":"粉砂岩",
                    "top_m":144.24,
                    "bot_m":150.0,
                    "description":"灰色，粉砂状结构，块状构造，泥质胶结，均匀层理，含少量炭化植物碎屑，半坚硬。"
                },
                {
                    "strata_name":"细粒砂岩",
                    "top_m":150.0,
                    "bot_m":168.5,
                    "description":"灰色，细粒砂状结构，厚层状构造，成分以石英为主，长石次之，云母及暗色矿物少量，分选中等，泥质胶结，均匀层理，半坚硬。"
                },
                {
                    "strata_name":"泥岩",
                    "top_m":168.5,
                    "bot_m":174.81,
                    "description":"灰色-深灰色，泥质结构，块状构造，断口平坦状，含少量炭化植物碎屑 及茎叶化石，半坚硬。"
                },
                {
                    "strata_name":"5煤",
                    "top_m":174.81,
                    "bot_m":175.08,
                    "description":"黑褐色，块状，暗煤为主，丝炭次之，夹亮煤条带，暗淡光泽，呈褐色 条痕，断口平坦状，风化易碎，污手，局部见有黄铁矿薄膜，属暗淡型煤，半坚硬。其中有泥岩夹矸4层，炭质泥岩夹矸3层。"
                }
            ]
        }
    ]
}


```

In [19]:
import json
import os
import pandas as pd
import numpy as np

from pandas.api.types import is_numeric_dtype, is_string_dtype

In [3]:
drill_list_file_path = "./data/Xi/xi-coords-selected.csv"
survey_line_file_path = "./data/Xi/line/"


list_survey_line_file_name = os.listdir(survey_line_file_path)
df_drill = pd.read_csv(drill_list_file_path, header=0, index_col=0)

list_drill_name = df_drill.index.tolist()

In [4]:
# 文件夹下所有的文件名称
ith_survey_line = 0

ith_survey_line_file_path = (
    survey_line_file_path + list_survey_line_file_name[ith_survey_line]
)
df_ith_survey_line = pd.read_csv(ith_survey_line_file_path, header=None, index_col=None)

In [ ]:
# check the columns of the dataframe
# 1. check if the number of columns is divisible by 6
cols_num = df_ith_survey_line.shape[1]
j = 0
df_drill_stratum = df_ith_survey_line.iloc[:, j * 6 : j * 6 + 6]
df_drill_stratum= df_drill_stratum.dropna(how="all")

# 2. check if the drill is in the list of drill names
drill_name = df_drill_stratum[0][0]
# json_string = ""

if drill_name in list_drill_name:

    drill_location = df_drill.loc[drill_name]

    coord_x = drill_location["coord_x"]
    coord_y = drill_location["coord_y"]
    elevation = drill_location["elevation"]

    string_drill_location = f'"drill_name":"drill-{drill_name}","coord_x":{coord_x:.3f},"coord_y":{coord_y:.3f},"elevation":{elevation:.3f}'

    stratum_num = df_drill_stratum.shape[0]

    # 验证地层高度信息的有效性，是否上下层相衔接
    x_rounded = np.round((df_drill_stratum[3]-df_drill_stratum[4]).values,3)
    y = np.zeros(stratum_num)
    y[1:]=df_drill_stratum[3].values[:-1]
    y_rounded = np.round(y,3)
    equal_elements = x_rounded == y_rounded
    all_equal = np.all(equal_elements)

    if all_equal:
        depth = df_drill_stratum[3][-1]
        string_drill_depth = f'"depth":{depth:.3f}'
        
        pass
    # iterate through each stratum and validate the data
    # remove the qoutes and space from the text data

    # save the json file

True


In [ ]:

if cols_num%6 ==0:
    drills_num  = int(cols_num/6)
    for j in range(drills_num):

        df_drill_stratum = df_ith_survey_line.iloc[:, j * 6 : j * 6+6].dtypes.to_list()
        
        if is_string_dtype(df_drill_stratum[0]):
            pass

In [ ]:
# stratum_dtype = [
#     dtype("O"),
#     dtype("O"),
#     dtype("O"),
#     dtype("float64"),
#     dtype("float64"),
#     dtype("O"),
# ]